In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import json
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
df = pd.read_csv(
    "https://mycs714awsbucket.s3.ca-central-1.amazonaws.com/amazon_reviews_us_Digital_Video_Download_v1_00.tsv",
    delimiter="\t",
    on_bad_lines='skip'
)

In [3]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,12190288,R3FU16928EP5TC,B00AYB1482,668895143,Enlightened: Season 1,Digital_Video_Download,5,0,0,N,Y,I loved it and I wish there was a season 3,I loved it and I wish there was a season 3... ...,2015-08-31
1,US,30549954,R1IZHHS1MH3AQ4,B00KQD28OM,246219280,Vicious,Digital_Video_Download,5,0,0,N,Y,As always it seems that the best shows come fr...,As always it seems that the best shows come fr...,2015-08-31
2,US,52895410,R52R85WC6TIAH,B01489L5LQ,534732318,After Words,Digital_Video_Download,4,17,18,N,Y,Charming movie,"This movie isn't perfect, but it gets a lot of...",2015-08-31
3,US,27072354,R7HOOYTVIB0DS,B008LOVIIK,239012694,Masterpiece: Inspector Lewis Season 5,Digital_Video_Download,5,0,0,N,Y,Five Stars,excellant this is what tv should be,2015-08-31
4,US,26939022,R1XQ2N5CDOZGNX,B0094LZMT0,535858974,On The Waterfront,Digital_Video_Download,5,0,0,N,Y,Brilliant film from beginning to end,Brilliant film from beginning to end. All of t...,2015-08-31


In [4]:
df = df[["customer_id", "product_id", "star_rating", "product_title"]]

In [5]:
customers = df["customer_id"].value_counts()
products = df["product_id"].value_counts()

quantiles = [
    0,
    0.01,
    0.02,
    0.03,
    0.04,
    0.05,
    0.1,
    0.25,
    0.5,
    0.75,
    0.9,
    0.95,
    0.96,
    0.97,
    0.98,
    0.99,
    1,
]
print("customers\n", customers.quantile(quantiles))
print("products\n", products.quantile(quantiles))

customers
 0.00       1.0
0.01       1.0
0.02       1.0
0.03       1.0
0.04       1.0
0.05       1.0
0.10       1.0
0.25       1.0
0.50       1.0
0.75       2.0
0.90       4.0
0.95       5.0
0.96       6.0
0.97       7.0
0.98       9.0
0.99      13.0
1.00    2704.0
Name: count, dtype: float64
products
 0.00        1.00
0.01        1.00
0.02        1.00
0.03        1.00
0.04        1.00
0.05        1.00
0.10        1.00
0.25        1.00
0.50        3.00
0.75        9.00
0.90       31.00
0.95       73.00
0.96       95.00
0.97      130.00
0.98      199.00
0.99      386.67
1.00    32790.00
Name: count, dtype: float64


In [6]:
customers = customers[customers >= 5]
products = products[products >= 10]

reduced_df = df.merge(pd.DataFrame({"customer_id": customers.index})).merge(
    pd.DataFrame({"product_id": products.index})
)

In [7]:
customers = reduced_df["customer_id"].value_counts()
products = reduced_df["product_id"].value_counts()

In [8]:
import numpy as np
customer_index = pd.DataFrame(
    {"customer_id": customers.index, "user": np.arange(customers.shape[0])}
)
product_index = pd.DataFrame({"product_id": products.index, "item": np.arange(products.shape[0])})

reduced_df = reduced_df.merge(customer_index).merge(product_index)
reduced_df.head()


,customer_id,product_id,star_rating,product_title,user,item
0,27072354,B008LOVIIK,5,Masterpiece: Inspector Lewis Season 5,9933,106
1,16030865,B008LOVIIK,5,Masterpiece: Inspector Lewis Season 5,481,106
2,44025160,B008LOVIIK,5,Masterpiece: Inspector Lewis Season 5,33290,106
3,18602179,B008LOVIIK,5,Masterpiece: Inspector Lewis Season 5,2144,106
4,14424972,B008LOVIIK,5,Masterpiece: Inspector Lewis Season 5,36659,106


In [9]:

test_df = reduced_df.groupby("customer_id").last().reset_index()

train_df = reduced_df.merge(
    test_df[["customer_id", "product_id"]],
    on=["customer_id", "product_id"],
    how="outer",
    indicator=True,
)
train_df = train_df[(train_df["_merge"] == "left_only")]


In [10]:
!pip install mxnet
!pip install gluon


In [11]:
!pip install numpy==1.23.5
import mxnet as mx
from mxnet import gluon, nd, np
from mxnet.metric import MSE

In [12]:
batch_size = 1024

train = gluon.data.ArrayDataset(
    nd.array(train_df["user"].values, dtype=np.float32),
    nd.array(train_df["item"].values, dtype=np.float32),
    nd.array(train_df["star_rating"].values, dtype=np.float32),
)
test = gluon.data.ArrayDataset(
    nd.array(test_df["user"].values, dtype=np.float32),
    nd.array(test_df["item"].values, dtype=np.float32),
    nd.array(test_df["star_rating"].values, dtype=np.float32),
)

train_iter = gluon.data.DataLoader(
    train, shuffle=True, num_workers=4, batch_size=batch_size, last_batch="rollover"
)
test_iter = gluon.data.DataLoader(
    train, shuffle=True, num_workers=4, batch_size=batch_size, last_batch="rollover"
)

In [13]:
class MFBlock(gluon.HybridBlock):
    def __init__(self, max_users, max_items, num_emb, dropout_p=0.5):
        super(MFBlock, self).__init__()

        self.max_users = max_users
        self.max_items = max_items
        self.dropout_p = dropout_p
        self.num_emb = num_emb

        with self.name_scope():
            self.user_embeddings = gluon.nn.Embedding(max_users, num_emb)
            self.item_embeddings = gluon.nn.Embedding(max_items, num_emb)

            self.dropout_user = gluon.nn.Dropout(dropout_p)
            self.dropout_item = gluon.nn.Dropout(dropout_p)

            self.dense_user = gluon.nn.Dense(num_emb, activation="relu")
            self.dense_item = gluon.nn.Dense(num_emb, activation="relu")

    def hybrid_forward(self, F, users, items):
        a = self.user_embeddings(users)
        a = self.dense_user(a)

        b = self.item_embeddings(items)
        b = self.dense_item(b)

        predictions = self.dropout_user(a) * self.dropout_item(b)
        predictions = F.sum(predictions, axis=1)
        return predictions

In [14]:

num_embeddings = 64

net = MFBlock(
    max_users=customer_index.shape[0],
    max_items=product_index.shape[0],
    num_emb=num_embeddings,
    dropout_p=0.5,
)

In [16]:
ctx = mx.gpu()
net.collect_params().initialize(mx.init.Xavier(magnitude=60), ctx=ctx, force_reinit=True)
net.hybridize()

# Set optimization parameters
opt = "sgd"
lr = 0.02
momentum = 0.9
wd = 0.0

trainer = gluon.Trainer(
    net.collect_params(), opt, {"learning_rate": lr, "wd": wd, "momentum": momentum}
)

MXNetError: Traceback (most recent call last):
  File "../src/ndarray/ndarray.cc", line 1297
MXNetError: GPU is not enabled